<a href="https://colab.research.google.com/github/Danie261/Trading-Algoritmico/blob/main/Backtesting_Strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BACKTESTING

In [ ]:
!pip install talib-binary
!pip install yfinance
!pip install backtesting

In [163]:
import talib as ta
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

ticker="GOOG"
df=yf.download(ticker,"2018-1-1")

def backtest(name):
  bt=Backtest(df, name, cash=10000, commission=0.002, exclusive_orders=True, trade_on_close=True)
  output=bt.run()
  bt.plot()

  print()
  print("*"*11, "BACKTESTING STRATEGY", "*"*11)
  print()
  print(output)

[*********************100%***********************]  1 of 1 completed


##'Four Indicators' Strategy

In [ ]:
class four_indicators(Strategy):

    t=100
    t_cci=12
    t_atr=14
    t_wr=11

    def init(self):

        self.sma=self.I(ta.SMA, self.data.Close, self.t)
        self.cci=self.I(ta.CCI, self.data.High, self.data.Low, self.data.Close, self.t_cci)
        self.atr=self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, self.t_atr)
        self.willr=self.I(ta.WILLR, self.data.High, self.data.Low, self.data.Close, self.t_wr)
    
    def next(self):

        price=self.data["Close"][-1]

        if self.data.Close>self.sma and self.willr<-85 and self.cci<-100 and self.atr>0.002:

            self.buy(sl=price*0.85, tp=price*1.3)

        elif self.data.Close<self.sma and self.willr>-15 and self.cci>100 and self.atr>0.002:

            self.sell(sl=price*1.2, tp=price*0.75)

backtest(four_indicators)

##'Weighted Average' Strategy

In [ ]:
class weighted_average(Strategy):

    t1=85
    t2=75
    t_ema=5

    def init(self):

        self.wma1=self.I(ta.WMA,self.data.Close, self.t1)
        self.wma2=self.I(ta.WMA,self.data.Close, self.t2)
        self.ema=self.I(ta.EMA, self.data.Close, self.t_ema)

    def next(self):

        price=self.data["Close"][-1]

        if self.wma1>self.wma2:
            
            if crossover(self.ema, self.wma1):

                self.buy()
            
            elif crossover(self.wma2, self.ema):

                self.sell(sl=price*1.2, tp=price*0.8)

        else:

            if crossover(self.ema, self.wma2):

                self.buy()

            elif crossover(self.wma1, self.ema):

                self.sell(sl=price*1.2, tp=price*0.8)

backtest(weighted_average)

##'SMA Cross' Strategy

In [ ]:
class SmaCross(Strategy):

    t1=10
    t2=20
    t_rsi=14

    def init(self):
        
        self.sma1=self.I(ta.SMA, self.data.Close, self.t1)
        self.sma2=self.I(ta.SMA, self.data.Close, self.t2)

    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.sma1, self.sma2):

            self.buy(sl=price*0.9, tp=price*1.2)

        elif crossover(self.sma2, self.sma1):
          
            self.sell(sl=price*1.2, tp=price*0.8)

backtest(SmaCross)

##'EMA Cross' Strategy

In [ ]:
class EmaCross(Strategy):

    t1=10
    t2=20

    def init(self):

        self.ema1=self.I(ta.EMA, self.data.Close, self.t1)
        self.ema2=self.I(ta.EMA, self.data.Close, self.t2)

    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.ema1, self.ema2):

            self.buy(sl=price*0.9, tp=price*1.1)

        elif crossover(self.ema2, self.ema1):

            self.sell(sl=price*1.1, tp=price*0.9)

backtest(EmaCross)

##'Close Cross' Strategy

In [ ]:
class CloseCross(Strategy):

    t2=20
    
    def init(self):

        self.sma=self.I(ta.SMA, self.data.Close, self.t2)

    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.data.Close, self.sma):

            self.buy(sl=price*0.85, tp=price*1.1)

        elif crossover(self.sma, self.data.Close):

            self.sell(sl=price*1.3, tp=price*0.75)

backtest(CloseCross)

##'Bollinger Bands' Strategy

In [ ]:
class BBands(Strategy):

    t=20

    def init(self):
        
        self.uband, self.mband, self.lband=self.I(ta.BBANDS, self.data.Close, self.t, 2,2,0)
        
    def next(self):
        
        price=self.data["Close"][-1]

        if crossover(self.data.Close, self.uband):

            self.sell(sl=price*1.2, tp=price*0.85) 
            
        elif crossover(self.lband, self.data.Close):  

            self.buy(sl=price*0.85, tp=price*1.1)

backtest(BBands)

##'Cross RSI' Strategy

In [ ]:
class Cross_RSI(Strategy):
    
    t=100
    t_rsi=14

    def init(self):

         self.sma = self.I(ta.SMA, self.data.Close, self.t)
         self.rsi=self.I(ta.RSI, self.data.Close, self.t_rsi)
        
    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.data.Close, self.sma) and self.rsi>50:

            self.buy()

        elif crossover(self.sma, self.data.Close) and self.rsi<50:

            self.sell(sl=price*1.1, tp=price*0.9)

backtest(Cross_RSI)

##'MACD and RSI' Strategy

In [ ]:
class macd(Strategy):
    
    t=30
    t_sma=200
    t_f=12
    t_s=26
    t_rsi=30

    def init(self):

        self.macd, self.signal, self.hist=self.I(ta.MACD, self.data.Close, self.t_f, self.t_s, 9)
        self.sma=self.I(ta.SMA, self.data.Close, self.t)
        self.sma2=self.I(ta.SMA, self.data.Close, self.t_sma)
        self.rsi=self.I(ta.RSI, self.data.Close, self.t_rsi)

    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.data.Close, self.sma) and self.sma>self.sma2 and self.macd>0 and self.rsi>50:

            self.buy()

        elif crossover(self.sma, self.data.Close) and self.sma<self.sma2 and self.macd<0 and self.rsi<50:

            self.sell(sl=price*1.3, tp=price*0.75)

backtest(macd)

##'CCI and RSI' Strategy

In [ ]:
class cci_rsi(Strategy):

    t_sma=150
    t=20

    def init(self):
        
        self.sma=self.I(ta.SMA, self.data.Close, self.t_sma)
        self.cci=self.I(ta.CCI, self.data.High, self.data.Low, self.data.Close, self.t)
        self.rsi=self.I(ta.RSI, self.data.Close, self.t)

    def next(self):

        price=self.data["Close"][-1]

        if crossover(self.data.Close, self.sma) and self.cci>0 and self.rsi>50:

            self.buy()

        elif crossover(self.sma, self.data.Close) and self.cci<0 and self.rsi<50:

            self.sell(sl=price*1.1, tp=price*0.9)

backtest(cci_rsi)